> ### **Interpolated String Handlers** in C#
>    Version *C# 10.0*

> **Interpolated String Handlers**: You can create a type that builds the resulting string from an interpolated string expression.   
>
>   In C# 10.0, an interpolated string handler is a type that processes the placeholder expression in an interpolated string. W
>   Without a custom handler, placeholders are processed similar to String.Format. 
>   Each placeholder is formatted as text, and then the components are concatenated to form the resulting string. 
>
>   We can write a handler for any scenario where you use information about the resulting string.

>  Here are two examples
> 
>  1. *Logging libraries*: Depending on the configured log level, the work to construct a log message isn’t needed.   
>         If logging is off, the work to construct a string from an interpolated string expression isn’t needed.                                                                                                                                                                                                     
>         The message is never printed, so any string concatenation can be skipped. In addition, any expressions used in the placeholders, including generating stack traces, doesn’t need to be done.  
>         An interpolated string handler can determine if the formatted string will be used, and only perform the necessary work if needed.
>
>   2. *Fixed-length buffer*: We can process the interpolated strings to fill a fixed-length buffer, and stop processing once that buffer length is reached.
        We may have a tabular format, and each placeholder must have a fixed length. A custom handler can enforce that, rather than forcing all client code to conform.

In [ ]:
//LogInterpolatedStringHandler: This handler builds a log message based on the log level and the logger object. It can skip unnecessary work when the log level is too low or the logger is disabled 
#r "nuget: Microsoft.Extensions.Logging"
using Microsoft.Extensions.Logging;

public enum LogLevel
{
    Off,
    Critical,
    Error,
    Warning,
    Information,
    Trace
}
public interface ILogger{

   void  LogMessage(LogLevel level, string msg);
}
public class Logger :ILogger
{
    public LogLevel EnabledLevel { get; init; } = LogLevel.Error;

    public void LogMessage(LogLevel level, string msg)
    {
        if (EnabledLevel < level) return;
        Console.WriteLine(msg);
    }
}

[System.Runtime.CompilerServices.InterpolatedStringHandler]
public class LogInterpolatedStringHandler 
{
    private readonly ILogger _logger;
    private readonly LogLevel _logLevel;

    public LogInterpolatedStringHandler(ILogger logger, LogLevel logLevel)
    {
        _logger = logger;
        _logLevel = logLevel;
    }

    public void AppendLiteral(string literal) { }

    public void AppendFormatted<T>(T value, string format, IFormatProvider formatProvider)
    {
       // if (_logger.IsEnabled(_logLevel)){
            _logger.LogMessage(_logLevel, value.ToString());
        //}
    }

    public string ToString(IFormatProvider formatProvider) => string.Empty;
}

var logger = new Logger() { EnabledLevel = LogLevel.Warning };
var time = DateTime.Now;

logger.LogMessage(LogLevel.Error, $"Error Level. CurrentTime: {time}. This is an error. It will be printed.");
logger.LogMessage(LogLevel.Trace, $"Trace Level. CurrentTime: {time}. This won't be printed.");
logger.LogMessage(LogLevel.Warning, "Warning Level. This warning is a string, not an interpolated string expression.");

> Example :2 Adding Caller Information

In [ ]:
using System.Runtime.CompilerServices;
[InterpolatedStringHandler]
public readonly ref struct LogInterpolatedStringHandler  
{  
  private readonly StringBuilder _builder;   

  public LogInterpolatedStringHandler (int literalLength, int formattedCount, [CallerFilePath] string filePath = "", [CallerLineNumber] int lineNumber = 0)  
  {  
    _builder = new StringBuilder (literalLength);   
    _builder.Append (filePath);  
    _builder.Append (':');  
    _builder.Append (lineNumber);  
  }   

  public void AppendLiteral (string s)  
  {  
    _builder.Append (s);  
  }   

  public void AppendFormatted<T> (T t)  
  {  
    _builder.Append (t);  
  }   

  internal string GetFormattedText () => _builder.ToString (); 
}
static void LogMessage (LogInterpolatedStringHandler handler) 
{ 
  Console.WriteLine (handler.GetFormattedText ()); 
} 
  LogMessage ($"string is {s}, int is {i}"); 


# Continue learning

There are plenty more resources out there to learn!

> [⏩ Next Module - Global Using Directives](72.GlobalUsingDirectives.ipynb)
>
> [⏪ Last Module - Improvements Of StructureTypes](70.ImprovementsOfStructureTypes.ipynb)
>
> [Reference - interpolated-string-handler](https://learn.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-10#interpolated-string-handler)    
> [Reference - interpolated-string-handler](https://learn.microsoft.com/en-us/dotnet/csharp/whats-new/tutorials/interpolated-string-handler)
> [Reference - C#-version-10](https://learn.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-10)  